In [128]:
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm, tqdm_notebook
import os
import time
import glob
import shutil
import os
import random
from PIL import Image
import cv2
import PythonMagick

In [166]:
df = pd.read_json('scryfall-oracle-cards.json')
df.head()

,all_parts,arena_id,artist,border_color,card_faces,cmc,collector_number,color_identity,color_indicator,colors,...,set,set_name,set_search_uri,set_uri,story_spotlight,tcgplayer_id,toughness,type_line,uri,watermark
0,NaN,NaN,Terese Nielsen,white,NaN,3.0,319,[],NaN,[],...,7ed,Seventh Edition,https://api.scryfall.com/cards/search?order=se...,https://api.scryfall.com/sets/230f38aa-9511-4d...,False,3094.0,NaN,Artifact,https://api.scryfall.com/cards/86bf43b1-8d4e-4...,NaN
1,NaN,NaN,Steven Belledin,black,NaN,1.0,71,[U],NaN,[U],...,m14,Magic 2014,https://api.scryfall.com/cards/search?order=se...,https://api.scryfall.com/sets/e03ee1c0-ecd2-4f...,False,69965.0,NaN,Enchantment â€” Aura,https://api.scryfall.com/cards/7050735c-b232-4...,NaN
2,NaN,NaN,John Matson,white,NaN,2.0,100,[U],NaN,[U],...,9ed,Ninth Edition,https://api.scryfall.com/cards/search?order=se...,https://api.scryfall.com/sets/e70c8572-4732-4e...,False,12835.0,2,Creature â€” Bird,https://api.scryfall.com/cards/036ef8c9-72ac-4...,NaN
3,NaN,NaN,Ron Spencer,black,NaN,2.0,47,[U],NaN,[U],...,ulg,Urza's Legacy,https://api.scryfall.com/cards/search?order=se...,https://api.scryfall.com/sets/78ced61b-4b8a-4b...,False,6412.0,1,Creature â€” Sponge,https://api.scryfall.com/cards/b125d1e7-5d9b-4...,NaN
4,NaN,NaN,Thomas M. Baxa,black,NaN,5.0,107,[R],NaN,[R],...,scg,Scourge,https://api.scryfall.com/cards/search?order=se...,https://api.scryfall.com/sets/5133c3a1-1412-4c...,False,10927.0,NaN,Sorcery,https://api.scryfall.com/cards/feeee859-f64a-4...,NaN


In [167]:
print(df.shape)
#df.drop_duplicates(subset='oracle_id', inplace=True)
#print(df.shape)

(19607, 59)


In [168]:
print(df.columns)

Index(['all_parts', 'arena_id', 'artist', 'border_color', 'card_faces', 'cmc',
       'collector_number', 'color_identity', 'color_indicator', 'colors',
       'digital', 'edhrec_rank', 'flavor_text', 'foil', 'frame',
       'frame_effect', 'full_art', 'games', 'hand_modifier', 'highres_image',
       'id', 'illustration_id', 'image_uris', 'lang', 'layout', 'legalities',
       'life_modifier', 'loyalty', 'mana_cost', 'mtgo_foil_id', 'mtgo_id',
       'multiverse_ids', 'name', 'nonfoil', 'object', 'oracle_id',
       'oracle_text', 'oversized', 'power', 'prints_search_uri', 'promo',
       'rarity', 'related_uris', 'released_at', 'reprint', 'reserved',
       'rulings_uri', 'scryfall_set_uri', 'scryfall_uri', 'set', 'set_name',
       'set_search_uri', 'set_uri', 'story_spotlight', 'tcgplayer_id',
       'toughness', 'type_line', 'uri', 'watermark'],
      dtype='object')


In [169]:
url_list = [jsd for jsd in df['image_uris'].values]
#print(url_list[0])
http_list = []
for url in url_list:
    try:
        http_list.append(url.get('art_crop'))
    except:
        http_list.append(np.nan)
print(len(url_list))
print(len(http_list))
print(http_list[:10])

19607
19607
['https://img.scryfall.com/cards/art_crop/en/7ed/319.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/m14/71.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/9ed/100.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/ulg/47.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/scg/107.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/me1/139.jpg?1535856219', 'https://img.scryfall.com/cards/art_crop/front/2/8/280f2a85-1900-460b-a768-164fc2dea636.jpg?1549414102', 'https://img.scryfall.com/cards/art_crop/en/ody/254.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/arb/104.jpg?1517813031', 'https://img.scryfall.com/cards/art_crop/en/m19/311.jpg?1531860983']


In [170]:
query_df = df[['color_identity', 'oracle_id']]
query_df['url'] = http_list
query_df.dropna(inplace=True)
query_df['id'] = [str(image_url.split('?')[-1]) for image_url in query_df['url']]
print(query_df.shape)
query_df.tail()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/panda

(19469, 4)


,color_identity,oracle_id,url,id
19602,[W],fff6f2d0-22b5-41d0-99dc-bc817c3bd166,https://img.scryfall.com/cards/art_crop/en/s99...,1517813031
19603,[B],fffa9334-3576-4f70-9605-2ad062cdbc69,https://img.scryfall.com/cards/art_crop/en/hou...,1517813031
19604,"[R, W]",fffaa634-42a0-4038-b027-24f28754fec7,https://img.scryfall.com/cards/art_crop/en/dom...,1535147949
19605,"[G, R]",fffdc2ac-bde4-4e4c-a5bd-0e6c6e49ad91,https://img.scryfall.com/cards/art_crop/en/hou...,1517813031
19606,[W],ffff90c3-63c4-4dee-a21d-6b2b113f4f80,https://img.scryfall.com/cards/art_crop/en/plc...,1517813031


In [171]:
print(query_df['url'].iloc[2000])

https://img.scryfall.com/cards/art_crop/en/ons/233.jpg?1517813031


In [172]:
print(query_df['url'].loc[query_df['id'] == '1551144270'])

Series([], Name: url, dtype: object)


In [173]:
query_df['filename'] = ['{}_{}_{}.jpg'.format(str(orid), str(cardid), color) for orid, cardid, color in zip(query_df['oracle_id'], query_df['id'], query_df['color_identity'])]

'''
print(query_df.head())
current_files = os.listdir('images')
print(current_files[:10])
query_df = query_df.loc[query_df['filename'].isin(current_files)]
'''
print(query_df.shape)
query_df.head()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


(19469, 5)


,color_identity,oracle_id,url,id,filename
0,[],0004ebd0-dfd6-4276-b4a6-de0003e94237,https://img.scryfall.com/cards/art_crop/en/7ed...,1517813031,0004ebd0-dfd6-4276-b4a6-de0003e94237_151781303...
1,[U],0006faf6-7a61-426c-9034-579f2cfcfa83,https://img.scryfall.com/cards/art_crop/en/m14...,1517813031,0006faf6-7a61-426c-9034-579f2cfcfa83_151781303...
2,[U],000d5588-5a4c-434e-988d-396632ade42c,https://img.scryfall.com/cards/art_crop/en/9ed...,1517813031,000d5588-5a4c-434e-988d-396632ade42c_151781303...
3,[U],000e5d65-96c3-498b-bd01-72b1a1991850,https://img.scryfall.com/cards/art_crop/en/ulg...,1517813031,000e5d65-96c3-498b-bd01-72b1a1991850_151781303...
4,[R],00181784-9213-492a-ba8c-2028969b049e,https://img.scryfall.com/cards/art_crop/en/scg...,1517813031,00181784-9213-492a-ba8c-2028969b049e_151781303...


In [174]:
print(len(list(set(query_df['oracle_id']))))

19469


In [175]:
query_df['counter'] = 1
id_df = query_df['counter'].groupby(query_df['id']).sum().reset_index()
id_df.sort_values('counter', ascending=False,inplace=True)
id_df.head()

C:\Users\Wolfgang\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,counter
0,1517813031,13396
2573,1530592713,3
4205,1541002937,2
4185,1541002878,2
4181,1541002865,2


In [147]:
query_df['url'].loc[query_df['id'] == '1530592568']

128026    https://img.scryfall.com/cards/art_crop/en/roe...
130005    https://img.scryfall.com/cards/art_crop/en/wwk...
Name: url, dtype: object

In [177]:
dir_location = 'images'


pbar = tqdm_notebook(total=len(query_df['url'].values))
for i, image_url in enumerate(query_df['url'].values):
    if i >= 3254: # incase we get booted we can restart from last image index
        img_data = requests.get(image_url).content
        filename = image_url.split('?')[-1]
        color = query_df['color_identity'].iloc[i]
        with open(os.path.join(dir_location, query_df['filename'].iloc[i]), 'wb') as handler:
            handler.write(img_data)
        time.sleep(0.1)
    pbar.update()
pbar.close()


















































Exception ignored in: <bound method tqdm.__del__ of  95%|█████████▌| 226000/237057 [36:34:01<24:02,  7.66it/s]>
Traceback (most recent call last):
  File "C:\Users\Wolfgang\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 889, in __del__
    self.close()
  File "C:\Users\Wolfgang\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 1095, in close
    self._decr_instances(self)
  File "C:\Users\Wolfgang\Anaconda3\lib\site-packages\tqdm\_tqdm.py", line 454, in _decr_instances
    cls.monitor.exit()
  File "C:\Users\Wolfgang\Anaconda3\lib\site-packages\tqdm\_monitor.py", line 52, in exit
    self.join()
  File "C:\Users\Wolfgang\Anaconda3\lib\threading.py", line 1053, in join
    raise RuntimeError("cannot join current thread")
RuntimeError: cannot join current thread


In [129]:
## searches for duplicates in the training dataset ##
## using raster hashes~~~~~~ ##

train_dir = 'images'
duplicate_img_dir= 'duplicate_images'

def is_similar(image1, image2):
    return image1.shape == image2.shape and not(np.bitwise_xor(image1,image2).any())

filelist = os.listdir(train_dir)
filelist = [os.path.join(train_dir, f) for f in filelist]

for i, file in enumerate(filelist):
    print('On File {}/{}:'.format(i, len(filelist)))
    this_file = PythonMagick.Image(file).signature()
    pbar = tqdm_notebook(total=len(range(i+1, len(filelist))))
    for j in range(i+1, len(filelist)):
        next_file = PythonMagick.Image(filelist[j]).signature()
        #if is_similar(this_file, next_file):
        if this_file == next_file:
            print('Similar file found: {}'.format(filelist[j]))
            try:
                os.rename(filelist[j], os.path.join(duplicate_img_dir, os.path.basename(filelist[j])))
                print('Removal success: {}'.format(filelist[j]))
            except WindowsError:
                print('Removal Failure: {}'.format(filelist[j]))
                try:
                    os.remove(filelist[j])
                    print('Deletion success: {}'.format(filelist[j]))
                except WindowsError:
                    print('Error deleting {}'.format(filelist[j]))
            j -= 1
        pbar.update()
    pbar.close()













  1%|          | 2212/186898 [02:01<2:30:49, 20.41it/s]

On File 0/186899:


KeyboardInterrupt: 

In [3]:
import numpy as np
a = np.random.normal(size=(4, 42, 1024))
a = np.transpose(a, (0,2,1))
b = np.random.normal(size=(42, 1024))
r = np.tensordot(a, b, [3, 0])
print(r.shape)

IndexError: tuple index out of range